In [3]:
# The following code is coppied and adapted from the following webpage:
# https://www.geeksforgeeks.org/emotion-detection-using-bidirectional-lstm/
# The attention layer is coppied and adapted from the following page:
# https://zhuanlan.zhihu.com/p/29201491?utm_id=0

In [4]:
# Importing the required libraries
import keras
import numpy as np
from keras.models import Sequential,Model
from keras.layers import Dense,Bidirectional
from nltk.tokenize import word_tokenize,sent_tokenize
from keras.layers import *
from sklearn.model_selection import cross_val_score,train_test_split
import nltk
import pandas as pd
import tensorflow as tf


nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\4K\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
df = pd.read_csv('text_emotion_isear_edit2.csv', header=1)

FileNotFoundError: [Errno 2] No such file or directory: 'text_emotion_isear_edit2.csv'

In [ ]:
#Changed a bit as now we have a DataFrame
df.drop(df[df['SIT'] == '[ No response.]'].index, inplace = True)
#let's get rid of anger and disgust to have a try
df = df[~df['EMOTION'].isin(['anger', 'disgust'])]
print (df['EMOTION'].value_counts() )

number_emotions = df['EMOTION'].unique()

In [ ]:

#make all contexts (sentences) in dataset a list
texts =df["SIT"]

#HIGHTLIGHT
#Convert each text value to lowercase using the .str.lower() method
lower_texts = texts.str.lower()

# Each  sentence in feel_arr is tokenized by the help of work tokenizer.
# If I have a sentence - 'I am happy'. 
# After word tokenizing it will convert into- ['I','am','happy']
lower_texts = [word_tokenize(sent) for sent in lower_texts]

#Now I added some codes to see how many words it has for the longest text
longest_text = max(lower_texts, key=len)
print (longest_text)
print("Sublist with the most elements:", len(longest_text))

In [ ]:

# Defined a function padd in which each sentence length is fixed to 200.
# If length is less than 200 , then the word- '<padd>' is append
def padd(arr):
    for i in range(200-len(arr)):
        arr.append('<pad>')
    return arr[:200]
   
# call the padd function for each sentence in feel_arr
for i in range(len(lower_texts)):
    lower_texts[i]=padd(lower_texts[i])
 

# Glove vector contains a 50 dimensional vector corresponding 
# to each word in dictionary.
vocab_f = 'glove.6B.50d.txt'
 
# embeddings_index is a dictionary which contains the mapping of
# word with its corresponding 50d vector.
embeddings_index = {}
with open(vocab_f, encoding='utf8') as f:
    for line in f:
        # splitting each line of the glove.6B.50d in a list of 
        # items- in which the first element is the word to be embedded,
        # and from second to the end of line contains the 50d vector.
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
         
# Embedding each word of the feel_arr
embedding_texts = []
# Iterate through each sub-list (sentences)
for word_list in lower_texts:
    # Initialize an empty list to store the embedding vectors for each word
    word_vectors = []
    for word in word_list:
        if word in embeddings_index:
            word_vector = embeddings_index[word]
            word_vectors.append(word_vector)
        else:
            # if the word to be embedded is '<padd>' append 0 fifty times
            word_vector = np.zeros(50, dtype='float32')
            word_vectors.append(word_vector)
    embedding_texts.append(word_vectors)


In [ ]:
#embeeding_texts cannot be a list, for the website he made it NumPy-array
X = np.array(embedding_texts)

#One-Hot Encoding for all emotions, 
#instead of just give them numbers, by doing so we will have seven outputs
Y = pd.get_dummies(df['EMOTION'], prefix='emotion')

# Split into train and test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=0.1,random_state=1)

In [ ]:
class BiLSTMModel:
    def __init__(self):
        self.model = Sequential()
        self.model.add(Input(shape=(200, 50)))
        
        self.model.add(Bidirectional(LSTM(64, input_shape=(200, 50))))

        self.model.add(Dropout(0.2))
        
        self.model.add(Dense(5, activation='softmax'))
        self.model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
 
    def fit(self, X, Y, epochs, batch_size,verbose):
        self.model.fit(X, Y, epochs=epochs, batch_size=batch_size,verbose=verbose)
 
    def evaluate(self, X, Y, batch_size,verbose):
        return self.model.evaluate(X, Y, batch_size=batch_size,verbose=verbose)
 
    def predict(self, X):
        return self.model.predict(X)

In [ ]:
@keras.saving.register_keras_serializable()
class Attention(Layer):
    def __init__(self, attention_size, **kwargs):
        self.attention_size = attention_size
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        # W: (EMBED_SIZE, ATTENTION_SIZE)
        # b: (ATTENTION_SIZE, 1)
        # u: (ATTENTION_SIZE, 1)
        self.W = self.add_weight(name="W_{:s}".format(self.name),
                                 shape=(input_shape[-1], self.attention_size),
                                 initializer="glorot_normal",
                                 trainable=True)
        self.b = self.add_weight(name="b_{:s}".format(self.name),
                                 shape=(input_shape[1], 1),
                                 initializer="zeros",
                                 trainable=True)
        self.u = self.add_weight(name="u_{:s}".format(self.name),
                                 shape=(self.attention_size, 1),
                                 initializer="glorot_normal",
                                 trainable=True)
        super(Attention, self).build(input_shape)

    def call(self, x, mask=None):
        # input: (BATCH_SIZE, MAX_TIMESTEPS, EMBED_SIZE)
        # et: (BATCH_SIZE, MAX_TIMESTEPS, ATTENTION_SIZE)
        et = K.tanh(K.dot(x, self.W) + self.b)
        # at: (BATCH_SIZE, MAX_TIMESTEPS)
        at = K.softmax(K.squeeze(K.dot(et, self.u), axis=-1))
        if mask is not None:
            at *= K.cast(mask, K.floatx())
        # ot: (BATCH_SIZE, MAX_TIMESTEPS, EMBED_SIZE)
        atx = K.expand_dims(at, axis=-1)
        ot = atx * x
        # output: (BATCH_SIZE, EMBED_SIZE)
        output = K.sum(ot, axis=1)
        return output

    def compute_mask(self, input, input_mask=None):
        return None

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [ ]:
from tensorflow.keras.layers import Layer, Bidirectional, LSTM, Input, Dense
import tensorflow as tf
import numpy as np
import tensorflow.keras.backend as K



input_layer = Input(shape=(200, 50))

# 创建一个Bidirectional LSTM层
lstm_layer = Bidirectional(LSTM(64, return_sequences=True))(input_layer)

# 创建一个注意力层
attention = Attention(5)(lstm_layer)

# 在注意力层之后添加其他层，例如全连接层
output_layer = tf.keras.layers.Dense(5, activation="softmax")(attention)

# 创建模型
model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])


In [ ]:
#model = BiLSTMModel()
# Take a look of the model layers summarry
model.summary()